In [2]:
# pip install tmdbsimple
# pip install python-dotenv

import pandas as pd

datapath = "data/"


In [4]:
#MOVIE METADATA

headers_movie_metadata = ["Wikipedia Movie ID", "Freebase Movie ID", "Movie name", "Movie release date",
                          "Movie box office revenue", "Movie runtime", "Movie languages", "Movie countries",
                          "Movie genres"]

movie_metadata = pd.read_csv('../data/movie.metadata.tsv', sep="\t", names=headers_movie_metadata)

#movie_metadata.count()
movie_metadata.head(145)
#movie_metadata[movie_metadata["Movie release date"]]

,Wikipedia Movie ID,Freebase Movie ID,Movie name,Movie release date,Movie box office revenue,Movie runtime,Movie languages,Movie countries,Movie genres
0,975900,/m/03vyhn,Ghosts of Mars,2001-08-24,14010832.0,98.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/01jfsb"": ""Thriller"", ""/m/06n90"": ""Science..."
1,3196793,/m/08yl5d,Getting Away with Murder: The JonBenét Ramsey ...,2000-02-16,NaN,95.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/02n4kr"": ""Mystery"", ""/m/03bxz7"": ""Biograp..."
2,28463795,/m/0crgdbh,Brun bitter,1988,NaN,83.0,"{""/m/05f_3"": ""Norwegian Language""}","{""/m/05b4w"": ""Norway""}","{""/m/0lsxr"": ""Crime Fiction"", ""/m/07s9rl0"": ""D..."
3,9363483,/m/0285_cd,White Of The Eye,1987,NaN,110.0,"{""/m/02h40lc"": ""English Language""}","{""/m/07ssc"": ""United Kingdom""}","{""/m/01jfsb"": ""Thriller"", ""/m/0glj9q"": ""Erotic..."
4,261236,/m/01mrr1,A Woman in Flames,1983,NaN,106.0,"{""/m/04306rv"": ""German Language""}","{""/m/0345h"": ""Germany""}","{""/m/07s9rl0"": ""Drama""}"
...,...,...,...,...,...,...,...,...,...
140,3550323,/m/09kzfd,Things to Do in Denver When You're Dead,1995-10,529677.0,115.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/01jfsb"": ""Thriller"", ""/m/0lsxr"": ""Crime F..."
141,1072373,/m/043j2x,Minsaara Kanavu,1997-01-14,NaN,148.0,"{""/m/07c9s"": ""Tamil Language""}","{""/m/03rk0"": ""India""}","{""/m/03q4nz"": ""World cinema"", ""/m/04t36"": ""Mus..."
142,1765938,/m/05v5ws,Convoy,1978,45000000.0,111.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/03btsm8"": ""Action/Adventure"", ""/m/02kdv5l..."
143,7460227,/m/0kv80y,The Bridge,2006-09-01,NaN,69.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/07s9rl0"": ""Drama""}"


In [12]:
movie_metadata[movie_metadata['Movie name'] == "The Gangsters"]["Movie release date"][5]

'1913-05-29'

In [7]:

#from dotenv import load_dotenv
#load_dotenv()
import os

%load_ext dotenv
%dotenv
TMDB_API_KEY = os.environ.get("TMDB_API_KEY")



The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [45]:
# use TVDB to populate missing revenue data : "Movie box office revenue
missing_revenue_data_count = movie_metadata['Movie box office revenue'].isna().sum()

import tmdbsimple as tmdb

# load API key
tmdb.API_KEY=TMDB_API_KEY
tmdb.REQUESTS_TIMEOUT = 0.3  # seconds, for both connect and read

#movie = tmdb.Find(4273140)
#response = movie.info()

movie_metadata_TMDB = movie_metadata.copy()

import re
from datetime import datetime

for index, row in movie_metadata_TMDB.iterrows():
    print(index)
    print(row)
    
    search = tmdb.Search()
    response = search.movie(query=row["Movie name"])
    
    # convert dataframe release to datetime
    movie_release_date = datetime.strptime(row["Movie release date"],"%Y-%m-%d").date()
    
    # convert query elements to datetime
    date_list_converted = [datetime.strptime(each_date['release_date'], "%Y-%m-%d").date() for each_date in search.results]
    
    # create list of differences in time
    differences = [abs(movie_release_date - each_date) for each_date in date_list_converted]
    minimum_index = differences.index(min(differences))  # Index of the closest match
    match = search.results[minimum_index]
    #print(f"Closest match: {match['title']} (Release Date: {match['release_date']})")
    
    # add info in dataframe about the movie

    movie_metadata_TMDB.loc[index,'TMDB_id'] = match['id']
    movie_metadata_TMDB.loc[index,'TMDB_original_language'] = match['original_language']
    movie_metadata_TMDB.loc[index,'TMDB_original_title'] = match['original_title']
    movie_metadata_TMDB.loc[index,'TMDB_overview'] = match['overview']
    movie_metadata_TMDB.loc[index,'TMDB_popularity'] = match['popularity']
    movie_metadata_TMDB.loc[index,'TMDB_release_date'] = match['release_date']
    movie_metadata_TMDB.loc[index,'TMDB_title'] = match['title']
    movie_metadata_TMDB.loc[index,'TMDB_vote_average'] = match['vote_average']
    movie_metadata_TMDB.loc[index,'TMDB_vote_count'] = match['vote_count']
    
    
movie_metadata_TMDB    
#df.loc[index,'d'] = np.random.randint(0, 10)
    


0
Wikipedia Movie ID                                                     975900
Freebase Movie ID                                                   /m/03vyhn
Movie name                                                     Ghosts of Mars
Movie release date                                                 2001-08-24
Movie box office revenue                                           14010832.0
Movie runtime                                                            98.0
Movie languages                            {"/m/02h40lc": "English Language"}
Movie countries                     {"/m/09c7w0": "United States of America"}
Movie genres                {"/m/01jfsb": "Thriller", "/m/06n90": "Science...
Name: 0, dtype: object


,Wikipedia Movie ID,Freebase Movie ID,Movie name,Movie release date,Movie box office revenue,Movie runtime,Movie languages,Movie countries,Movie genres,TMDB_id,TMDB_original_language,TMDB_original_title,TMDB_overview,TMDB_popularity,TMDB_release_date,TMDB_title,TMDB_vote_average,TMDB_vote_count
0,975900,/m/03vyhn,Ghosts of Mars,2001-08-24,14010832.0,98.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/01jfsb"": ""Thriller"", ""/m/06n90"": ""Science...",10016.0,en,Ghosts of Mars,"In 2176, a Martian police unit is sent to pick...",17.28,2001-08-24,Ghosts of Mars,5.123,980.0
1,3196793,/m/08yl5d,Getting Away with Murder: The JonBenét Ramsey ...,2000-02-16,NaN,95.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/02n4kr"": ""Mystery"", ""/m/03bxz7"": ""Biograp...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,28463795,/m/0crgdbh,Brun bitter,1988,NaN,83.0,"{""/m/05f_3"": ""Norwegian Language""}","{""/m/05b4w"": ""Norway""}","{""/m/0lsxr"": ""Crime Fiction"", ""/m/07s9rl0"": ""D...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,9363483,/m/0285_cd,White Of The Eye,1987,NaN,110.0,"{""/m/02h40lc"": ""English Language""}","{""/m/07ssc"": ""United Kingdom""}","{""/m/01jfsb"": ""Thriller"", ""/m/0glj9q"": ""Erotic...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,261236,/m/01mrr1,A Woman in Flames,1983,NaN,106.0,"{""/m/04306rv"": ""German Language""}","{""/m/0345h"": ""Germany""}","{""/m/07s9rl0"": ""Drama""}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81736,35228177,/m/0j7hxnt,Mermaids: The Body Found,2011-03-19,NaN,120.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/07s9rl0"": ""Drama""}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
81737,34980460,/m/0g4pl34,Knuckle,2011-01-21,NaN,96.0,"{""/m/02h40lc"": ""English Language""}","{""/m/03rt9"": ""Ireland"", ""/m/07ssc"": ""United Ki...","{""/m/03bxz7"": ""Biographical film"", ""/m/07s9rl0...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
81738,9971909,/m/02pygw1,Another Nice Mess,1972-09-22,NaN,66.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/06nbt"": ""Satire"", ""/m/01z4y"": ""Comedy""}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
81739,913762,/m/03pcrp,The Super Dimension Fortress Macross II: Lover...,1992-05-21,NaN,150.0,"{""/m/03_9r"": ""Japanese Language""}","{""/m/03_3d"": ""Japan""}","{""/m/06n90"": ""Science Fiction"", ""/m/0gw5n2f"": ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Processing:   0%|          | 4/81741 [00:00<3:48:23,  5.96it/s]

Error at index 2: time data '1988' does not match format '%Y-%m-%d'
Error at index 3: time data '1987' does not match format '%Y-%m-%d'


Processing:   0%|          | 6/81741 [00:00<3:48:04,  5.97it/s]

Error at index 4: time data '1983' does not match format '%Y-%m-%d'
Error at index 5: time data '' does not match format '%Y-%m-%d'


Processing:   0%|          | 8/81741 [00:01<3:47:03,  6.00it/s]

...

Processing:   0%|          | 22/81741 [00:03<3:53:23,  5.84it/s]

Error at index 20: time data '1930' does not match format '%Y-%m-%d'


Processing:   0%|          | 24/81741 [00:04<3:52:20,  5.86it/s]

Error at index 22: time data '2009' does not match format '%Y-%m-%d'
Error at index 23: min() arg is an empty sequence


Processing:   0%|          | 27/81741 [00:04<3:53:23,  5.84it/s]

In [ ]:





search = tmdb.Search()
response = search.movie(query='Harry Potter and the Deathly Hallows – Part 2')
for s in search.results:
    print(s['title'], s['id'], s['release_date'], s['popularity'])

print(search.results)


In [34]:
# Read new file created in other script "add_TMDB_movie_metadata"


movie_metadata_TMDB = pd.read_csv('movie_metadata_TMDB.csv',header=0) 

#movie_metadata.count()
#movie_metadata_TMDB.head(500).sample(25)
movie_metadata_TMDB.head(10)

,Wikipedia Movie ID,Freebase Movie ID,Movie name,Movie release date,Movie box office revenue,Movie runtime,Movie languages,Movie countries,Movie genres,TMDB_id,TMDB_original_language,TMDB_original_title,TMDB_overview,TMDB_popularity,TMDB_release_date,TMDB_title,TMDB_vote_average,TMDB_vote_count
0,975900,/m/03vyhn,Ghosts of Mars,2001-08-24,14010832.0,98.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/01jfsb"": ""Thriller"", ""/m/06n90"": ""Science...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3196793,/m/08yl5d,Getting Away with Murder: The JonBenét Ramsey ...,2000-02-16,NaN,95.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/02n4kr"": ""Mystery"", ""/m/03bxz7"": ""Biograp...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,28463795,/m/0crgdbh,Brun bitter,1988,NaN,83.0,"{""/m/05f_3"": ""Norwegian Language""}","{""/m/05b4w"": ""Norway""}","{""/m/0lsxr"": ""Crime Fiction"", ""/m/07s9rl0"": ""D...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,9363483,/m/0285_cd,White Of The Eye,1987,NaN,110.0,"{""/m/02h40lc"": ""English Language""}","{""/m/07ssc"": ""United Kingdom""}","{""/m/01jfsb"": ""Thriller"", ""/m/0glj9q"": ""Erotic...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,261236,/m/01mrr1,A Woman in Flames,1983,NaN,106.0,"{""/m/04306rv"": ""German Language""}","{""/m/0345h"": ""Germany""}","{""/m/07s9rl0"": ""Drama""}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,13696889,/m/03cfc81,The Gangsters,1913-05-29,NaN,35.0,"{""/m/06ppq"": ""Silent film"", ""/m/02h40lc"": ""Eng...","{""/m/09c7w0"": ""United States of America""}","{""/m/02hmvc"": ""Short Film"", ""/m/06ppq"": ""Silen...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,18998739,/m/04jcqvw,The Sorcerer's Apprentice,2002,NaN,86.0,"{""/m/02h40lc"": ""English Language""}","{""/m/0hzlz"": ""South Africa""}","{""/m/0hqxf"": ""Family Film"", ""/m/01hmnh"": ""Fant...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,10408933,/m/02qc0j7,Alexander's Ragtime Band,1938-08-16,3600000.0,106.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/04t36"": ""Musical"", ""/m/01z4y"": ""Comedy"", ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,9997961,/m/06_y2j7,Contigo y aquí,1974,NaN,NaN,"{""/m/06nm1"": ""Spanish Language""}","{""/m/0jgd"": ""Argentina""}","{""/m/04t36"": ""Musical"", ""/m/07s9rl0"": ""Drama"",...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2345652,/m/075f66,City of the Dead,1960,NaN,76.0,"{""/m/02h40lc"": ""English Language""}","{""/m/07ssc"": ""United Kingdom""}","{""/m/03npn"": ""Horror"", ""/m/0fdjb"": ""Supernatur...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
# merge CSV files
import pandas as pd
md_rest = pd.read_csv('movie_metadata_TMDBrest.csv')
md_28000 = pd.read_csv('movie_metadata_TMDB28000.csv')
md_11000 = pd.read_csv('movie_metadata_TMDB11407.csv')



C:\Users\gusta\AppData\Local\Temp\ipykernel_58440\2788347056.py:4: DtypeWarning: Columns (10,11,12,14,15) have mixed types. Specify dtype option on import or set low_memory=False.
  md_28000 = pd.read_csv('movie_metadata_TMDB28000.csv')
C:\Users\gusta\AppData\Local\Temp\ipykernel_58440\2788347056.py:5: DtypeWarning: Columns (10,11,12,14,15) have mixed types. Specify dtype option on import or set low_memory=False.
  md_11000 = pd.read_csv('movie_metadata_TMDB11407.csv')


In [40]:
big = md_rest['TMDB_original_language'].isna().sum()
med = md_28000['TMDB_original_language'].isna().sum()
small = md_11000['TMDB_original_language'].isna().sum()


# test join
partone = md_28000.combine_first(md_11000)
df = partone.combine_first(md_rest).copy()
df['TMDB_original_language'].isna().sum()

df['Wikipedia Movie ID'].count()

print(f"Added TMDB ID to {df['TMDB_id'].count()} movies. Total movies before {df['Wikipedia Movie ID'].count()}. \n Percentage populated: {round(df['TMDB_id'].count()/df['Wikipedia Movie ID'].count()*100,2)}%")

df.to_csv('movie_metadata_TMDBid.csv')



Added TMDB ID to 68944 movies. Total movies before 81741. 
 Percentage populated: 84.34%


In [1]:
# read v2 TMDB
import pandas as pd
import numpy as np

df = pd.read_csv('movie_metadata_TMDB_v2_COMBINED_DONE.csv.csv')
OG_rev = df['Movie box office revenue'].loc[0:30000].isnull().sum()
#df_test = df.replace(0,np.nan,inplace=True).copy()
df_v2 = df.copy()
df_v2.replace(0,np.nan,inplace=True)

NEW_rev = df_v2['TMDB_revenue'].loc[0:30000].isnull().sum()

print("Original revenue NaN:", OG_rev)
print("New revenue NaN:",NEW_rev)
print("Difference", OG_rev-NEW_rev)
#df[['TMDB_revenue','Movie box office revenue']].head(150)
df.head(150)
#df['TMDB_revenue'].isnull()

Original revenue NaN: 26299
New revenue NaN: 29941
Difference -3642


C:\Users\gusta\AppData\Local\Temp\ipykernel_11352\4192360628.py:5: DtypeWarning: Columns (22,23) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('movie_metadata_TMDB_v2.csv')


,Unnamed: 0,Wikipedia Movie ID,Freebase Movie ID,Movie name,Movie release date,Movie box office revenue,Movie runtime,Movie languages,Movie countries,Movie genres,...,TMDB_popularity,TMDB_release_date,TMDB_title,TMDB_vote_average,TMDB_vote_count,TMDB_runtime,TMDB_revenue,TMDB_budget,TMDB_IMDB_id,TMDB_genres
0,0,975900,/m/03vyhn,Ghosts of Mars,2001-08-24,14010832.0,98.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/01jfsb"": ""Thriller"", ""/m/06n90"": ""Science...",...,17.280,2001-08-24,Ghosts of Mars,5.123,980.0,NaN,NaN,NaN,NaN,NaN
1,1,3196793,/m/08yl5d,Getting Away with Murder: The JonBenét Ramsey ...,2000-02-16,NaN,95.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/02n4kr"": ""Mystery"", ""/m/03bxz7"": ""Biograp...",...,0.750,2000-02-16,Getting Away with Murder: The JonBenét Ramsey ...,8.000,1.0,NaN,NaN,NaN,NaN,NaN
2,2,28463795,/m/0crgdbh,Brun bitter,1988,NaN,83.0,"{""/m/05f_3"": ""Norwegian Language""}","{""/m/05b4w"": ""Norway""}","{""/m/0lsxr"": ""Crime Fiction"", ""/m/07s9rl0"": ""D...",...,0.600,1988-11-17,Hair of the Dog,0.000,0.0,NaN,NaN,NaN,NaN,NaN
3,3,9363483,/m/0285_cd,White Of The Eye,1987,NaN,110.0,"{""/m/02h40lc"": ""English Language""}","{""/m/07ssc"": ""United Kingdom""}","{""/m/01jfsb"": ""Thriller"", ""/m/0glj9q"": ""Erotic...",...,7.336,1987-06-19,White of the Eye,5.742,64.0,NaN,NaN,NaN,NaN,NaN
4,4,261236,/m/01mrr1,A Woman in Flames,1983,NaN,106.0,"{""/m/04306rv"": ""German Language""}","{""/m/0345h"": ""Germany""}","{""/m/07s9rl0"": ""Drama""}",...,2.397,1983-05-11,A Woman in Flames,5.300,13.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,174,6305925,/m/0g08ws,The Astronaut Farmer,2006-10-15,11130889.0,104.0,"{""/m/02h40lc"": ""English Language"", ""/m/06nm1"":...","{""/m/09c7w0"": ""United States of America""}","{""/m/06n90"": ""Science Fiction"", ""/m/09n5t_"": ""...",...,10.219,2007-02-23,The Astronaut Farmer,6.192,308.0,NaN,NaN,NaN,NaN,NaN
146,175,196176,/m/01bwgr,Straw Dogs,1971-11-03,11148828.0,118.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America"", ""/m/...","{""/m/01jfsb"": ""Thriller"", ""/m/07s9rl0"": ""Drama""}",...,13.956,1971-11-25,Straw Dogs,7.139,844.0,NaN,NaN,NaN,NaN,NaN
147,176,34454683,/m/0h_92lt,"Agent 3S3, Massacre in the Sun",1966-05-20,NaN,106.0,"{""/m/02bjrlw"": ""Italian Language""}","{""/m/0f8l9c"": ""France"", ""/m/06mkj"": ""Spain"", ""...","{""/m/0lsxr"": ""Crime Fiction"", ""/m/03k9fj"": ""Ad...",...,0.739,1966-05-20,"Agent 3S3, Massacre in the Sun",4.000,1.0,NaN,NaN,NaN,NaN,NaN
148,177,9944484,/m/04dld6,The Battle of Algiers,1966-09-03,NaN,121.0,"{""/m/064_8sq"": ""French Language"", ""/m/0jzc"": ""...","{""/m/03rjj"": ""Italy"", ""/m/0h3y"": ""Algeria""}","{""/m/0lsxr"": ""Crime Fiction"", ""/m/03q4nz"": ""Wo...",...,10.424,1966-09-08,The Battle of Algiers,7.912,660.0,NaN,NaN,NaN,NaN,NaN


In [32]:
# before merging, inspect datasets
import pandas as pd

og_df = pd.read_csv("../modified_data/movie_metadata_TMDB.csv")

new_df = pd.read_csv("movie_metadata_TMDB_v2_COMBINED_DONE.csv")


display(og_df)
display(new_df)
#display(new_df[['Movie name','TMDB_title']].sample(50))

,Unnamed: 0,Wikipedia Movie ID,Freebase Movie ID,Movie name,Movie release date,Movie box office revenue,Movie runtime,Movie languages,Movie countries,Movie genres,TMDB_id,TMDB_original_language,TMDB_original_title,TMDB_overview,TMDB_popularity,TMDB_release_date,TMDB_title,TMDB_vote_average,TMDB_vote_count
0,0,975900,/m/03vyhn,Ghosts of Mars,2001-08-24,14010832.0,98.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/01jfsb"": ""Thriller"", ""/m/06n90"": ""Science...",10016.0,en,Ghosts of Mars,"In 2176, a Martian police unit is sent to pick...",17.280,2001-08-24,Ghosts of Mars,5.123,980.0
1,1,3196793,/m/08yl5d,Getting Away with Murder: The JonBenét Ramsey ...,2000-02-16,NaN,95.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/02n4kr"": ""Mystery"", ""/m/03bxz7"": ""Biograp...",784579.0,en,Getting Away with Murder: The JonBenét Ramsey ...,Dramatization of the story behind the murder o...,0.750,2000-02-16,Getting Away with Murder: The JonBenét Ramsey ...,8.000,1.0
2,2,28463795,/m/0crgdbh,Brun bitter,1988,NaN,83.0,"{""/m/05f_3"": ""Norwegian Language""}","{""/m/05b4w"": ""Norway""}","{""/m/0lsxr"": ""Crime Fiction"", ""/m/07s9rl0"": ""D...",396302.0,no,Brun bitter,A stolen bicycle case ends with drunken detect...,0.600,1988-11-17,Hair of the Dog,0.000,0.0
3,3,9363483,/m/0285_cd,White Of The Eye,1987,NaN,110.0,"{""/m/02h40lc"": ""English Language""}","{""/m/07ssc"": ""United Kingdom""}","{""/m/01jfsb"": ""Thriller"", ""/m/0glj9q"": ""Erotic...",33592.0,en,White of the Eye,"In a wealthy and isolated desert community, a ...",7.336,1987-06-19,White of the Eye,5.742,64.0
4,4,261236,/m/01mrr1,A Woman in Flames,1983,NaN,106.0,"{""/m/04306rv"": ""German Language""}","{""/m/0345h"": ""Germany""}","{""/m/07s9rl0"": ""Drama""}",11192.0,de,Die flambierte Frau,"Eva, an upper-class housewife, frustratedly le...",2.397,1983-05-11,A Woman in Flames,5.300,13.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81736,81736,35228177,/m/0j7hxnt,Mermaids: The Body Found,2011-03-19,NaN,120.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/07s9rl0"": ""Drama""}",117124.0,en,Mermaids: The Body Found,A story that imagines how these real-world phe...,5.098,2011-03-19,Mermaids: The Body Found,4.500,20.0
81737,81737,34980460,/m/0g4pl34,Knuckle,2011-01-21,NaN,96.0,"{""/m/02h40lc"": ""English Language""}","{""/m/03rt9"": ""Ireland"", ""/m/07ssc"": ""United Ki...","{""/m/03bxz7"": ""Biographical film"", ""/m/07s9rl0...",44946.0,en,Knucklehead,A fight promoter deeply in debt to his crooked...,9.789,2010-10-22,Knucklehead,5.500,50.0
81738,81738,9971909,/m/02pygw1,Another Nice Mess,1972-09-22,NaN,66.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/06nbt"": ""Satire"", ""/m/01z4y"": ""Comedy""}",285337.0,en,Another Nice Mess,Nixon and Agnew played as Laurel and Hardy.,1.960,1972-08-23,Another Nice Mess,0.000,0.0
81739,81739,913762,/m/03pcrp,The Super Dimension Fortress Macross II: Lover...,1992-05-21,NaN,150.0,"{""/m/03_9r"": ""Japanese Language""}","{""/m/03_3d"": ""Japan""}","{""/m/06n90"": ""Science Fiction"", ""/m/0gw5n2f"": ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


,Unnamed: 0.1,Unnamed: 0,Wikipedia Movie ID,Freebase Movie ID,Movie name,Movie release date,Movie box office revenue,Movie runtime,Movie languages,Movie countries,...,TMDB_popularity,TMDB_release_date,TMDB_title,TMDB_vote_average,TMDB_vote_count,TMDB_runtime,TMDB_revenue,TMDB_budget,TMDB_IMDB_id,TMDB_genres
0,0,0,975900,/m/03vyhn,Ghosts of Mars,2001-08-24,14010832.0,98.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}",...,17.280,2001-08-24,Ghosts of Mars,5.123,980.0,98.0,14010832.0,28000000.0,tt0228333,"Action, Horror, Science Fiction"
1,1,1,3196793,/m/08yl5d,Getting Away with Murder: The JonBenét Ramsey ...,2000-02-16,NaN,95.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}",...,0.750,2000-02-16,Getting Away with Murder: The JonBenét Ramsey ...,8.000,1.0,60.0,NaN,0.0,tt0245916,"Drama, Crime"
2,2,2,28463795,/m/0crgdbh,Brun bitter,1988,NaN,83.0,"{""/m/05f_3"": ""Norwegian Language""}","{""/m/05b4w"": ""Norway""}",...,0.600,1988-11-17,Hair of the Dog,0.000,0.0,83.0,NaN,0.0,tt0094806,"Mystery, Crime, Drama"
3,3,3,9363483,/m/0285_cd,White Of The Eye,1987,NaN,110.0,"{""/m/02h40lc"": ""English Language""}","{""/m/07ssc"": ""United Kingdom""}",...,7.336,1987-06-19,White of the Eye,5.742,64.0,111.0,NaN,0.0,tt0094320,"Horror, Thriller"
4,4,4,261236,/m/01mrr1,A Woman in Flames,1983,NaN,106.0,"{""/m/04306rv"": ""German Language""}","{""/m/0345h"": ""Germany""}",...,2.397,1983-05-11,A Woman in Flames,5.300,13.0,106.0,NaN,0.0,tt0083949,Drama
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68939,68939,81735,32468537,/m/0crwd9y,Shadow Boxing 2,2007-10-18,NaN,132.0,"{""/m/06b_j"": ""Russian Language"", ""/m/02h40lc"":...","{""/m/06bnz"": ""Russia""}",...,4.421,2007-10-18,Revenge,5.607,28.0,135.0,11171900.0,6000000.0,tt0459759,"Action, Drama, Mystery"
68940,68940,81736,35228177,/m/0j7hxnt,Mermaids: The Body Found,2011-03-19,NaN,120.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}",...,5.098,2011-03-19,Mermaids: The Body Found,4.500,20.0,82.0,NaN,0.0,tt1816585,Science Fiction
68941,68941,81737,34980460,/m/0g4pl34,Knuckle,2011-01-21,NaN,96.0,"{""/m/02h40lc"": ""English Language""}","{""/m/03rt9"": ""Ireland"", ""/m/07ssc"": ""United Ki...",...,9.789,2010-10-22,Knucklehead,5.500,50.0,100.0,NaN,0.0,tt1524131,"Drama, Comedy"
68942,68942,81738,9971909,/m/02pygw1,Another Nice Mess,1972-09-22,NaN,66.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}",...,1.960,1972-08-23,Another Nice Mess,0.000,0.0,66.0,NaN,0.0,tt0362411,Comedy


In [10]:
# merge enrich_v2 datasets together

import pandas as pd
import numpy as np

md_done = pd.read_csv('movie_metadata_TMDB_v2_DONE.csv',index_col=0)
md_30000 = pd.read_csv('movie_metadata_TMDB_v2 - 30000.csv',index_col=0)
md_35000 = pd.read_csv('movie_metadata_TMDB_v2 - 35000.csv',index_col=0)

# test join
#partone = md_30000.combine_first(md_35000)
#display(md_30000)
#df = partone.combine_first(md_done).copy()

# merge 
temp_df = pd.merge(md_30000,md_35000,how='left')
#display(temp_df)
df = pd.merge(temp_df,md_done,how='left')

df['TMDB_revenue'].replace(0,np.nan,inplace=True) # NaN values are 0 in the revenue data collected. 0 is replaced by NaN
display(df)
print(df.columns)
# the combined data must now be combined with the original data

df.to_csv('movie_metadata_TMDB_v2_COMBINED_DONE.csv',index=False)



C:\Users\gusta\AppData\Local\Temp\ipykernel_6152\3810251486.py:7: DtypeWarning: Columns (22,23) have mixed types. Specify dtype option on import or set low_memory=False.
  md_30000 = pd.read_csv('movie_metadata_TMDB_v2 - 30000.csv',index_col=0)
C:\Users\gusta\AppData\Local\Temp\ipykernel_6152\3810251486.py:8: DtypeWarning: Columns (22,23) have mixed types. Specify dtype option on import or set low_memory=False.
  md_35000 = pd.read_csv('movie_metadata_TMDB_v2 - 35000.csv',index_col=0)


,Wikipedia Movie ID,Freebase Movie ID,Movie name,Movie release date,Movie box office revenue,Movie runtime,Movie languages,Movie countries,Movie genres,TMDB_id,...,TMDB_popularity,TMDB_release_date,TMDB_title,TMDB_vote_average,TMDB_vote_count,TMDB_runtime,TMDB_revenue,TMDB_budget,TMDB_IMDB_id,TMDB_genres
0,975900,/m/03vyhn,Ghosts of Mars,2001-08-24,14010832.0,98.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/01jfsb"": ""Thriller"", ""/m/06n90"": ""Science...",10016.0,...,17.280,2001-08-24,Ghosts of Mars,5.123,980.0,98.0,14010832.0,28000000.0,tt0228333,"Action, Horror, Science Fiction"
1,3196793,/m/08yl5d,Getting Away with Murder: The JonBenét Ramsey ...,2000-02-16,NaN,95.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/02n4kr"": ""Mystery"", ""/m/03bxz7"": ""Biograp...",784579.0,...,0.750,2000-02-16,Getting Away with Murder: The JonBenét Ramsey ...,8.000,1.0,60.0,NaN,0.0,tt0245916,"Drama, Crime"
2,28463795,/m/0crgdbh,Brun bitter,1988,NaN,83.0,"{""/m/05f_3"": ""Norwegian Language""}","{""/m/05b4w"": ""Norway""}","{""/m/0lsxr"": ""Crime Fiction"", ""/m/07s9rl0"": ""D...",396302.0,...,0.600,1988-11-17,Hair of the Dog,0.000,0.0,83.0,NaN,0.0,tt0094806,"Mystery, Crime, Drama"
3,9363483,/m/0285_cd,White Of The Eye,1987,NaN,110.0,"{""/m/02h40lc"": ""English Language""}","{""/m/07ssc"": ""United Kingdom""}","{""/m/01jfsb"": ""Thriller"", ""/m/0glj9q"": ""Erotic...",33592.0,...,7.336,1987-06-19,White of the Eye,5.742,64.0,111.0,NaN,0.0,tt0094320,"Horror, Thriller"
4,261236,/m/01mrr1,A Woman in Flames,1983,NaN,106.0,"{""/m/04306rv"": ""German Language""}","{""/m/0345h"": ""Germany""}","{""/m/07s9rl0"": ""Drama""}",11192.0,...,2.397,1983-05-11,A Woman in Flames,5.300,13.0,106.0,NaN,0.0,tt0083949,Drama
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68939,32468537,/m/0crwd9y,Shadow Boxing 2,2007-10-18,NaN,132.0,"{""/m/06b_j"": ""Russian Language"", ""/m/02h40lc"":...","{""/m/06bnz"": ""Russia""}","{""/m/01z02hx"": ""Sports"", ""/m/0lsxr"": ""Crime Fi...",56525.0,...,4.421,2007-10-18,Revenge,5.607,28.0,NaN,NaN,NaN,NaN,NaN
68940,35228177,/m/0j7hxnt,Mermaids: The Body Found,2011-03-19,NaN,120.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/07s9rl0"": ""Drama""}",117124.0,...,5.098,2011-03-19,Mermaids: The Body Found,4.500,20.0,NaN,NaN,NaN,NaN,NaN
68941,34980460,/m/0g4pl34,Knuckle,2011-01-21,NaN,96.0,"{""/m/02h40lc"": ""English Language""}","{""/m/03rt9"": ""Ireland"", ""/m/07ssc"": ""United Ki...","{""/m/03bxz7"": ""Biographical film"", ""/m/07s9rl0...",44946.0,...,9.789,2010-10-22,Knucklehead,5.500,50.0,NaN,NaN,NaN,NaN,NaN
68942,9971909,/m/02pygw1,Another Nice Mess,1972-09-22,NaN,66.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/06nbt"": ""Satire"", ""/m/01z4y"": ""Comedy""}",285337.0,...,1.960,1972-08-23,Another Nice Mess,0.000,0.0,NaN,NaN,NaN,NaN,NaN


Index(['Wikipedia Movie ID', 'Freebase Movie ID', 'Movie name',
       'Movie release date', 'Movie box office revenue', 'Movie runtime',
       'Movie languages', 'Movie countries', 'Movie genres', 'TMDB_id',
       'TMDB_original_language', 'TMDB_original_title', 'TMDB_overview',
       'TMDB_popularity', 'TMDB_release_date', 'TMDB_title',
       'TMDB_vote_average', 'TMDB_vote_count', 'TMDB_runtime', 'TMDB_revenue',
       'TMDB_budget', 'TMDB_IMDB_id', 'TMDB_genres'],
      dtype='object')
